---
title: Interactive Visualization Notebook of Key Results
short_title: Interactive Visualization
description: |
    This notebook showcases the ability to develop and package interactive visualizations with MyST and Curvenote
---

# Interactive Visualization Notebook of Key Results

This notebook creates interactive visualizations for exploring groundwater recharge patterns, river networks, and subsurface characteristics in California's Central Valley.

Reusable function can be found in ['notebooks/utils.py'](./utils.py)

## Setup and Imports

**Key Libraries:**

- **`xarray` & `rioxarray`**: Handle multi-dimensional raster data with geospatial awareness
- **`geopandas`**: Manage vector geospatial data (rivers, basins, points)
- **`ipyleaflet`**: Create interactive Leaflet.js maps in Jupyter
- **`ipywidgets`**: Build interactive UI controls (dropdowns, sliders)

In [1]:
import xarray as xr
import geopandas as gpd
from ipyleaflet import (
    GeoJSON,
    TileLayer,
)
import json
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.pyplot as plt
import numpy as np
from utils import (
    find_intersections,
    combine_rivers_gdf,
    DualMapController,
)

## Configuration

**Steps**:
- Define filepaths
- Unzip data generated in []('./1_resistivity_to_metric_maps.ipynb')
- Generate custom colorbars(that match the static file output in []('./1_resistivity_to_metric_maps.ipynb')), 
- Define color parameters (like colorlimits and labels), and map settings in a central location.

In [2]:
# Data Paths
# This data is generated in the first notebook and stored in `notebooks/data_export.zip`
export_dir = "./data_export_unzipped/"

In [3]:
# Unzip quietly (suppress output)
!unzip -qo data_export.zip -d {export_dir}

In [4]:
# Vector data
vector_rivers = f"{export_dir}/rivers.geojson"
vector_subbasin = f"{export_dir}/subbasin.geojson"
vector_brackish = f"{export_dir}/brackish.geojson"

# Raster/gridded data
metric_zarr_path = f"{export_dir}/consolidated_metric_output.zarr"

# Statistics
stats_path = f"{export_dir}/statistics_combined.json"

# define custom colormaps
colors = [
    (0, 0.5, 0),  # Green
    (1, 1, 1),
]  # White
positions = [0, 1]
green_r_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

colors = [
    (1, 1, 1),  # Green
    (0, 0.5, 0),
]  # White
positions = [0, 1]
green_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

# Extract RGB values from the predefined RdBu colormap
cmap = plt.get_cmap("RdBu")

# Create a list of colors by sampling the colormap
colors = [cmap(i) for i in range(cmap.N)]

# Create the custom colormap
custom_rdbu = LinearSegmentedColormap.from_list("custom_rdbu", np.array(colors)[20:-20][::-1])

# Color Parameters
param_dict = {
    "fraction_coarse": {
        "vmin": 0,
        "vmax": 100,
        "short_label": "FCD",
        "label": "Fraction of coarse-dominated sediment (%)",
        "cmap": custom_rdbu,
        "log_scale": False,
    },
    "path_length_norm": {
        "vmin": 1,
        "vmax": 1000,
        "short_label": "Normalized distance",
        "label": "Normalized path length",
        "cmap": green_r_map,
        "log_scale": True,
    },
    "path_to_no_flow": {
        "vmin": 0,
        "vmax": 20,
        "short_label": "Depth to no flow",
        "label": "Depth to shallowest no-flow unit\n or base surface (m)",
        "cmap": green_map,
        "log_scale": False,
    },
    "summary_metric": {
        "vmin": 0,
        "vmax": 1,
        "short_label": "Suitable",
        "label": "Summary Metric",
        "cmap": ListedColormap(["none", "green"]),  # Binary with transparency
        "log_scale": False,
    },
}

# Map Configuration
center = [37.66335291403956, -120.69523554193438]  # Central Valley center
zoom = 7  # Regional zoom level
basemap = None  # Use custom tile layers instead of default basemap
map_width = "500px"
map_height = "800px"

## Data Loading and Preprocessing

Load all datasets and prepare them for visualization by:
1. Simplifying river geometry via label values
2. Finding intersection points between rivers and the subbasins
3. All data is already simplyfied and reprojected to common crs (EPSG:4326) in []('./1_resistivity_to_metric_maps.ipynb')

In [5]:
# -------------------------------
# Load Vector Layers
# -------------------------------
brackish = gpd.read_file(vector_brackish)
rivers = gpd.read_file(vector_rivers)
# Combine river segments that share the same name
rivers = combine_rivers_gdf(rivers)
# TODO: QC this - some rivers have gaps and potential duplicate names

subbasins = gpd.read_file(vector_subbasin)

# Find points where rivers exit the basin boundary
river_intersections = find_intersections(rivers, subbasins)

# -------------------------------
# Load Raster Data (Consolidated Metrics)
# -------------------------------
ds = xr.open_zarr(metric_zarr_path)
ds = ds.drop_vars("spatial_ref")

# Mask out each land use type as a raster variable
# Generate boolean mask for each land type

land_use_id_to_names = {
    11: "Open Water",
    12: "Perennial Snow/Ice",
    21: "Developed, Open Space",
    22: "Developed",
    23: "Developed",
    24: "Developed",
    31: "Barren Land",
    41: "Forest",
    42: "Forest",
    43: "Forest",
    52: "Shrub/Scrub",
    71: "Herbaceous",
    81: "Hay/Pasture",
    82: "Cultivated Crops",
    90: "Wetlands",
    95: "Wetlands",
}

number_to_land_type_simple = {
    "Open Water": [11],
    "Perennial Snow/Ice": [12],
    "Developed, Open Space": [21],
    "Developed": [22, 23, 24],
    "Barren Land": [31],
    "Forest": [41, 42, 43],
    "Shrub/Scrub": [52],
    "Herbaceous": [71],
    "Hay/Pasture": [81],
    "Cultivated Crops": [82],
    "Wetlands": [90, 95],
}
land_use_colors = {
    "Open Water": "#4f9dfc",
    "Perennial Snow/Ice": "#bde3f7",
    "Developed, Open Space": "#fee8c8",
    "Developed": "#e34a33",
    "Barren Land": "#d9d9d9",
    "Forest": "#228b22",
    "Shrub/Scrub": "#d95f0e",
    "Herbaceous": "#e7e247",
    "Hay/Pasture": "#ffff99",
    "Cultivated Crops": "#f2b701",
    "Wetlands": "#41ab5d",
}

land_use_display_order = [
    "Cultivated Crops",
    "Herbaceous",
    "Shrub/Scrub",
    "Developed",
    "Developed, Open Space",
    "Open Water",
    "Wetlands",
    "Hay/Pasture",
    "Barren Land",
    "Forest",
]

for idx, (name, ids) in enumerate(number_to_land_type_simple.items()):
    var_name = f"land_use_{name}"
    color = land_use_colors[name]

    ds[var_name] = ds["land_use"].isin(ids).astype(int)
    param_dict[var_name] = {
        "vmin": 0,
        "vmax": 1,
        "short_label": var_name.replace("land_use_", ""),
        "label": "Land Use: " + var_name.replace("land_use_", ""),
        "cmap": ListedColormap(["none", color]),  # "none" for transparency!
        "log_scale": False,
    }

# Load land use stats
with open(stats_path, "r") as f:
    land_use_stats = json.load(f)

## Layer Creation

Create all map layers with associated styling.

In [6]:
# -------------------------------
# Basemap Tile Layers
# -------------------------------
# Hillshade elevation for terrain context
l_elevation = TileLayer(
    url="https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}",
    name="Elevation",
    opacity=1.0,
)

# -------------------------------
# Vector Layers (Lines & Polygons)
# -------------------------------
l_rivers = GeoJSON(
    data=rivers.__geo_interface__,
    style={"color": "blue", "weight": 1, "opacity": 0.7},
    name="Rivers",
)

l_subbasins = GeoJSON(
    data=subbasins.__geo_interface__,
    style={"color": "black", "weight": 1, "fill": False},
    name="Subbasins",
)

l_brackish = GeoJSON(
    data=brackish.__geo_interface__,
    style={
        "color": "black",
        "weight": 1,
        "fillColor": "gray",
        "fillOpacity": 1.0,
    },
    name="Brackish Areas",
)

# River exit/inflow points at basin boundary
intersections_layer = GeoJSON(
    data=river_intersections.__geo_interface__,
    point_style={
        "radius": 6,
        "color": "orange",
        "fillColor": "orange",
        "fillOpacity": 0.5,
        "width": 2,
    },
    name="River Exits",
)

## Synchronized Interactive Maps

Side-by-side interactive maps with synchronized views for comparative analysis.

### Architecture

**DualMapController Class:**
A custom controller that manages two linked maps with shared navigation and independent data layers.

**Key Features:**
1. **View Synchronization**: Zoom and pan are linked via `ipywidgets.jslink()`
2. **Independent Layers**: Each map displays different data for comparison
3. **Interactive Highlighting**: Click features to highlight across both maps
4. **Dynamic Data Updates**: Switch datasets and thresholds in real-time

### Interactive Workflows

**Subbasin Selection:**
1. Choose from dropdown → map zooms to bounds
2. Subbasin outline highlighted with orange dashed border
3. Highlight persists until new selection
4. "Home" button returns to default view

**River Selection:**
1. Click orange marker at river exit point
2. Full river geometry highlights in bright blue on both maps
3. Popup shows river name

**Dataset Exploration:**
1. Select metric from dropdown (right map only)
2. Adjust FCD threshold with slider
3. Right map updates raster overlay
4. Left map unchanged for reference comparison

In [7]:
# | label: fig:multi-maps
# -------------------------------
# Dual Map Visualization
# -------------------------------

# Initialize controller with statistics
controller = DualMapController(
    width=map_width,
    height=map_height,
    center=center,
    zoom=zoom,
    subbasins=subbasins,
    subbasin_column="Basin_Su_1",
    rivers_gdf=rivers,
    river_intersections_gdf=river_intersections,
    param_dict=param_dict,
)

# -------------------------------
# Assemble and Display Maps
# -------------------------------
# Add base vector layers to both maps
controller.add_layer(l_rivers)
controller.add_layer(l_subbasins)
controller.add_layer(l_brackish)

# Add scalar raster layers to each map
# Left map: Fraction Coarse Dominated (static reference - 2D variable, no controls)
controller.add_scalar_layer(ds, "fraction_coarse", map_side="left")

# Right map: Multiple variables with dropdown + slider for threshold dimension
controller.add_scalar_layer(
    ds,
    ["path_length_norm", "path_to_no_flow"],
    map_side="right",
)

# Add controls and interactive elements
controller.add_controls()  # Must be called AFTER adding base layers
controller.add_river_intersections(intersections_layer)
controller.create_subbasin_selector(center=center, zoom=zoom)
controller.display()

Output()

In [8]:
# | label: fig:multi-maps-summary
# -------------------------------
# Dual Map Visualization Summary
# -------------------------------

# Initialize controller
controller2 = DualMapController(
    width=map_width,
    height=map_height,
    center=center,
    zoom=zoom,
    subbasins=subbasins,
    subbasin_column="Basin_Su_1",
    rivers_gdf=rivers,
    river_intersections_gdf=river_intersections,
    param_dict=param_dict,
    stats_data=land_use_stats,
)

# -------------------------------
# Assemble and Display Maps
# -------------------------------
# Add base vector layers to both maps
controller2.add_layer(l_rivers)
controller2.add_layer(l_subbasins)
controller2.add_layer(l_brackish)

# Add scalar raster layers to each map
# Left map: Fraction Coarse Dominated (static reference - 2D variable, no controls)
controller2.add_scalar_layer(ds, "summary_metric", map_side="left")

# Right map: Multiple variables with dropdown + slider for threshold dimension
controller2.add_scalar_layer(
    ds,
    ["land_use_" + var for var in land_use_display_order],
    map_side="right",
)

# Add controls and interactive elements
controller2.add_controls()  # Must be called AFTER adding base layers
controller2.add_river_intersections(intersections_layer)
controller2.create_subbasin_selector(center=center, zoom=zoom)
controller2.display()

Output()